# Analyse over Wijn


## onderzoeksvragen
- In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?  
- Zijn de wijnen die met een hoge score beoordeeld zijn duurder dan wijnen met een lagere score?
- In hoeverre kan op basis van de chemische kenmerken voorspelt worden of het een witte of rode wijn is?
- Welke kernwoorden zijn typerend voor een hoog scorende wijn?  

Wij zullen deze onderzoeksvragen zo goed mogelijk proberen te beantwoorden.
Voor de beantwoording van deze vragen maken wij gebruik van het Data Science proces.  
Dit proces ziet er als volgt uit:
### Het data science proces
1. Data collection
2. Data processing
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication


Allereerst zullen we de benodigde libraries importeren.
Deze libraries zullen we gebruiken voor het analyseren en het visualiseren van de data.

In [ ]:
# data analysis libraries 
import numpy as np
import pandas as pd

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns


## Data Collection
De eerste dataset die we gaan gebruiken is aan ons aangeleverd. De dataset is een csv bestand en heet: 'redwine.csv'. In de dataset staat informatie en chemische kenmerken van wijnen in Portugal. 

Allereerst beginnen we met het importeren van de dataset en kijken we of de dataset correct is geïmporteerd door de eerste 5 regels op te vragen.

In [ ]:
# import the data files
df_red_wine = pd.read_csv("data/redwine.csv", sep=";")
df_red_wine.head()

### externe dataset
Aangezien onze eerste dataset alleen maar kenmerken van rode wijn bevat zijn wij gaan zoeken naar een dataset met kenmerken van witte wijn.  
  
De dataset komt van de volgende website: https://archive.ics.uci.edu/ml/datasets/wine+quality  
  
Ook deze dataset gaan we importeren om hierna te controleren of hij goed geïmporteerd is.

In [ ]:
df_white_wine = pd.read_csv("data/winequality-white.csv", sep=";")
df_white_wine.head()

## Data Processing
Aangezien de bestanden met de data al in een csv bestand staan kunnen deze direct in een dataframe worden ingelezen.  
Nu is het nog aan ons om te beslissen of er nog kolomnamen zijn die aangepast moeten worden en welke kolommen we gaan droppen.  
In totaal heeft deze dataset 22 kolommen. Om uit te zoeken welke kolommen essentieel zijn voor het verdere proces gaan we nu eerste kijken hoe de kolommen heten.

In [ ]:
df_red_wine.columns

Om goed te kunnen begrijpen met welke data we hier mee te maken hebben is hieronder voor iedere kolom de betekenis van de data die erin staat gegeven.  

- **id**  
    Een uniek nummer voor iedere rij
- **Country**  
    Het land waar de wijn vandaan komt
- **Description**  
    De beschrijving van de wijn
- **Designation**  
    De wijngaard waar tenminste 85% van de druiven vandaan komen
- **Points**  
    De hoeveelheid punten die de wijn heeft gekregen van de proever
- **Price**  
    De prijs van de wijn
- **Province**  
    De provincie waar de wijn
- **Taster_name**  
    De volledige naam van de proever
- **Title**  
    De titel die normaliter op het wijnetiket staat
- **Variety**  
    Het type druif dat gebruikt wordt
- **Winery**  
    Het bedrijf waar de wijn geproduceerd is
- **Fixed acidity**  
    
- **Volatile acidity**  
    
- **Citric acid**  
    
- **Residual sugar**  
    
- **Chlorides**  
    De hoeveelheid zout in de wijn
- **Free sulfur dioxide**  
    
- **Total sulfur dioxide**  
    
- **Density**  
    
- **pH**  
    Is een maat voor de zuurgraad van een waterige oplossing. De schaal gaat van 0 (zuur) tot 14 (basisch). De meeste wijnen zitten tussen de 2.8 en de 4.0
- **Sulphates (sulfites)**  
    
- **Alcohol**  
    Het percentage alcohol in de wijn

Voor het uitzoeken van de betekenis van deze termen/kolommen zijn de volgende bronnen gebruikt:  
- https://www.gall.nl/ontdek/wijn/de-zuurgraad-van-wijn/
- https://winecompliancealliance.com/vinyard-designation-on-a-wine-label/
- http://rstudio-pubs-static.s3.amazonaws.com/80458_5000e31f84df449099a872ccf40747b7.html


Bij deze dataset hebben we al de voorkennis dat al deze wijnen uit Portugal komen. Hierdoor hoeven we de kolom 'country' niet te gebruiken. Verder geeft 'variety' aan met wat voor druif we te maken hebben, waarschijnlijk zal dit ook maar één unieke waarde hebben.  
Voordat we deze kolommen weggooien checken we nog voor de zekerheid of onze aannames kloppen. 

In [ ]:
df_red_wine['country'].unique()

In [ ]:
df_red_wine['variety'].unique()

Onze aannames blijken te kloppen. We hebben hier twee keer te maken met een kolom met maar één unieke waarde. Deze kolommen kunnen dus gedropt worden.

In [ ]:
df_red_wine.drop(['country', 'variety'], axis=1, inplace=True)

Op basis van onze onderzoeksvragen kunnen we ook de kolommen 'Designation', 'winery' en 'Province' laten vallen. Dit omdat we voor de beantwoording van onze onderzoeksvragen geen onderscheid hoeven te maken tussen de verschillende provincies of de wijngaarden waar de wijn vandaan kan komen.

In [ ]:
df_red_wine.drop(['designation', 'province', 'winery'], axis=1, inplace=True)

Verder maken we van de kolom 'id' de index

In [ ]:
df_red_wine.set_index('id', inplace=True)
df_red_wine.head()

## Data cleaning
Nu de data bewerkt is kunnen we tot de volgende stap overgaan: het opschonen van de data. Hierbij gaan we op zoek naar missende en dubbele waarden, outliers en onvolkomenheden.  
Allereerst gaan we opzoek naar rijen waar waarden missen.

In [ ]:
len(df_red_wine)

In [ ]:
pd.isnull(df_red_wine).sum()

Hier zien we dat er 269 missende waarden zijn en dat die waarden zich allemaal in de 'price' kolom bevinden. Deze waarde hebben we echter wel voor één van de onderzoeksvragen. Hier hebben wij de afweging gemaakt om deze waarden te verwijderen.  
Deze keuze is gebaseerd op het feit dat we nu ongeveer 12% van de data weggooien en we dus nog genoeg data behouden om te onderzoeken of er een relatie zit tussen de prijs en de hoeveelheid punten van een fles wijn.  
We droppen dus alle rijen met missende waardes en tegelijkertijd verwijderen we duplicate rijen.

In [ ]:
# deleting the rows with empty values
df_red_wine.dropna(inplace=True)
# deleting the duplicates rows
df_red_wine.drop_duplicates(inplace=True)

chemische kenmerken filteren!!!!!!!!

## Data Exploration

In [ ]:
df_red_wine.describe()

Wat uitleg over bovenstaande tabel

### De wijnproevers
Aangezien we met wijnproevers te maken hebben is alle data over de punten die aan een wijn toegedeeld zijn subjectief. Allereerst willen we wat inzicht krijgen over het gedrag van deze groep bij de toekenning van de punten.  
  
Eerst gaan we kijken met hoeveel wijnproevers we in totaal te maken hebben.

In [ ]:
len(df_red_wine['taster_name'].unique())

We hebben dus met 18 verschillende wijnproevers te maken.  
Hierna gaan we kijken of de frequenties van de beoordelingen van de wijnproevers gelijk verdeeld zijn.

In [ ]:
df_red_wine['taster_name'].value_counts()

Uit de bovenstaande tabel valt te zien dat de frequenties van de beoordelingen niet evenredig verdeeld zijn. Zo valt te zien dat sommige proevers zeer veel beoordelingen hebben gegeven terwijl andere veel minder hebben gegeven.

In [ ]:
df_taster_index = df_red_wine.copy()
df_taster_index.set_index('taster_name', inplace=True)
df_taster_index.head()